In this tutorial, we firstly recall the *port-Hamiltonian systems (PHS)* formalism in parallel with a description of the `pyphs.PHSCore` object. Secondly, we build the `pyphs.PHSCore` object associated with the standard serial resistor-coil-capacitor (RLC) circuit.
<!-- TEASER_END -->

# Core PHS structure of the RLC circuit

Steps are:
1. prepare the *governing equations*,
2. instantiate a `pyphs.PHSCore` *object*,
3. add the *components* (storage, dissipative and source),
4. define the *PHS structure* matrices (serial connection),
5. set the *physical parameters*.

## 1. Governing equations
Following the reference [[1, §2.2]](http://www.mdpi.com/2076-3417/6/10/273/pdf), the RLC circuit is described as follows:
* $x_L$ is the coil flux so that $v_L = \frac{\mathrm d x_L}{\mathrm d  t}$ is the coil voltage, 
* $x_C$ is the electric charge associated with the capacitor so that $i_C = \frac{\mathrm d x_C}{\mathrm d  t}$ is the capacitor current, 
* $w_R= i_R$ is the resistor current, 
* $y=i_{\mathrm{out}}$ is the output current, and 
* $u=v_{\mathrm{out}}$ is the input voltage.

The constitutive laws are:
* the quadratic storage function $H(x_L, x_C)=\frac{x_L^2}{2L}+\frac{x_C^2}{2C},$ so that the coil current is $i_L=\frac{\partial H}{\partial x_L}=\frac{x_L}{L}$ and the capacitor voltage is $v_C=\frac{\partial H}{\partial x_C}=\frac{x_C}{C}$,
* the linear dissipation function $z_R(w_R)= R \,w_R = v_R$ with $v_R$ the resistor voltage. 

The Kirchhoff's laws for a serial connection read: 
* Kirchhoff's current law: $ v_L=-v_C-v_R-v_{\mathrm{out}}$,
* Kirchhoff's voltage law: $i_L = i_C = i_R = i_{\mathrm{out}}$.

This can be expressed in the *Port-Hamiltonian Systems* (PHS) formalism as:

$$\left(\begin{array}{c}\frac{\mathrm d x_L}{\mathrm d  t} \\ \frac{\mathrm d x_C}{\mathrm d  t}\\ \hline w_R\\ \hline y \end{array}\right)=\left(\begin{array}{cc|c|c}
0 & -1 & -1 & -1\\ 
1 & 0 & 0 & 0 \\ \hline
1 & 0 & 0 & 0 \\  \hline
1 & 0 & 0 & 0
\end{array}\right)\cdot \left(\begin{array}{c}\frac{\partial H}{\partial x_C}\\ \frac{\partial H}{\partial x_L}\\ \hline z_R\\ \hline u \end{array}\right) $$

#### Physical parameters
* $C=2\times 10^{-9}$F, 
* $L=50\times 10^{-3}$H, 
* $R = 10^3\Omega$.

## 2. Object instantiation

In [1]:
# Support for Python 2.x and Python 3.x
from __future__ import division, print_function, absolute_import

# load the pyphs.PHSCore class in the current namespace
from pyphs import PHSCore

# instantiate a pyphs.PHSCore object
core = PHSCore()

Now, `core` is an instance of `pyphs.PHSCore`:

In [2]:
print("isinstance(core, PHSCore) = {}".format(isinstance(core, PHSCore)))
print("core.x = {}".format(core.x))
print("core.dims.w() = {}".format(core.dims.w()))
print("core.J() = {}".format(core.J()))
print("core.Jxy() = {}".format(core.Jxy()))

isinstance(core, PHSCore) = True
core.x = []
core.dims.w() = 0
core.J() = Matrix(0, 0, [])
core.Jxy() = Matrix(0, 0, [])


## 3. Adding the components
### Defining symbols
The `pyphs` package is based on the `sympy` package to provide the symbolic manipulation of PHS structures. To declare symbols, we use the `core.symbols` method. As an example, we declare below the symbols associated with the coil: 
* the state $x_L$ (magnetic flux of the coil), and 
* the parameter $L$ (coil inductance).

In [3]:
xL, L = core.symbols(['xL', 'L'])

### Defining expressions
Now, the variables `xL` and `L` are instances of the `sympy.Symbol` object, with the assumption that quantities are real-valued only. Then, expressions can be defined with the standard `sympy` syntax. As an example, we define below the storage function associated with the coil $H_L(x_L)=\frac{x_L^2}{2L}$:

In [4]:
HL = xL**2/(2*L)

### the `core.add_storages` method
To include a storage component to a `PHSCore`, we make use of the `core.add_storages` method. As an example, the coil is added to the `core` object as follows:

In [5]:
core.add_storages(xL, HL)

For the capacitor:
* the state is $x_C$ (electric charge),
* the parameter is $C$ (electric capacitance), and 
* the storage function is $H_C(x_C)=\frac{x_C^2}{2C}$.

This component is added to the `core` object as follows: 

In [6]:
xC, C = core.symbols(['xC', 'C'])
HC = xC**2/(2*C)
core.add_storages(xC, HC)

Now, the state of the `core` object includes both `xL` and `xC`, and the storage function is given by the sum of `HL` and `HC`:

In [7]:
print("core.x = {}".format(core.x))
print("core.H = {}".format(core.H))

core.x = [xL, xC]
core.H = xL**2/(2*L) + xC**2/(2*C)


Notice the same results can be obtained with a single call to `add_storages` by 
1. defining a list of states:
```python
x = [xL, xC]
```
2. defining a total storage function:
```python
H = HL + HC
```
3. calling:
```python
core.add_storages(x, H)
```

### the `core.add_dissipations` method
To include a dissipative component to a `PHSCore`, we make use of the `core.add_dissipations` method. Recall the resistor is decribed by:
* the dissipative variable $w_R$ (resistor current),
* the parameter $R$ (electric resistance), and 
* the dissipation function $z_R(w_R)=R\,w_R$.

This component is added to the `core` object as follows: 

In [8]:
wR, R = core.symbols(['wR', 'R'])    # define sympy symbols
zR = R*wR                           # define sympy expression
core.add_dissipations(wR, zR)        # add dissipation to the `core` object

Now, the dissipative variable of the `core` object includes `wR` only:

In [9]:
core.w

[wR]

and the dissipation function is given by `zR` only:

In [10]:
core.z

[R*wR]

### the `core.add_ports` method
To include an external port to a `PHSCore` object, we make use of the `core.add_ports` method. Below, we define the external port with input $u=v_{\mathrm{out}}$ and output $y=i_{\mathrm{out}}$ (notice the symbols do not reflect the actual physical meaning of $u$ and $y$):

In [11]:
u, y = core.symbols(['vout', 'iout']) # define sympy symbols
core.add_ports(u, y)                  # add the port to the `core` object

## 4. Defining the interconnection structure
The interconnection structure of a PHS is defined by the matrix $\mathbf M$ structured as $$\mathbf M = \mathbf J- \mathbf R,$$ where 
* the skew-symmetric matrix $\mathbf J = \frac{1}{2}\left(\mathbf M- \mathbf M^\intercal\right) $ encodes the *conservative interconnection*, and 
* the symmetric positive definite matrix $\mathbf R = \frac{-1}{2}\left(\mathbf M + \mathbf M^\intercal\right)$ encodes the *dissipative interconnection*.

These matrices are decomposed in blocks as follows:
$$\mathbf M = \left( 
\begin{array}{lll}
\mathbf M_{\mathrm{xx}} & \mathbf M_{\mathrm{xw}} & \mathbf M_{\mathrm{xy}} \\
\mathbf M_{\mathrm{wx}} & \mathbf M_{\mathrm{ww}} & \mathbf M_{\mathrm{wy}} \\
\mathbf M_{\mathrm{yx}} & \mathbf M_{\mathrm{ww}} & \mathbf M_{\mathrm{yy}} \\
\end{array}\right), 
$$
$$
\mathbf J = \left( 
\begin{array}{lll}
\mathbf J_{\mathrm{xx}} & \mathbf J_{\mathrm{xw}} & \mathbf J_{\mathrm{xy}} \\
\mathbf J_{\mathrm{wx}} & \mathbf J_{\mathrm{ww}} & \mathbf J_{\mathrm{wy}} \\
\mathbf J_{\mathrm{yx}} & \mathbf J_{\mathrm{ww}} & \mathbf J_{\mathrm{yy}} \\
\end{array}\right), \quad\mathbf R = \left( 
\begin{array}{lll}
\mathbf R_{\mathrm{xx}} & \mathbf R_{\mathrm{xw}} & \mathbf R_{\mathrm{xy}} \\
\mathbf R_{\mathrm{wx}} & \mathbf R_{\mathrm{ww}} & \mathbf R_{\mathrm{wy}} \\
\mathbf R_{\mathrm{yx}} & \mathbf R_{\mathrm{ww}} & \mathbf R_{\mathrm{yy}} \\
\end{array}\right).$$

For the above description of the RLC circuit, the matrices are $$\mathbf M = \mathbf J = \left(\begin{array}{cc|c|c}
0 & -1 & -1 & -1\\ 
1 & 0 & 0 & 0 \\ \hline
1 & 0 & 0 & 0 \\  \hline
1 & 0 & 0 & 0
\end{array}\right),$$ 
that is, $\mathbf R=0$. Each block of the matrix $\mathbf J$ can be defined as a `numpy.array` as follows:

In [12]:
import numpy

Jxx = numpy.array([[0, -1],
                   [1, 0]])
core.set_Jxx(Jxx)

Jxw = numpy.array([[-1],
                   [0]])
core.set_Jxw(Jxw)

Jxy = numpy.array([[-1],
                   [0]])
core.set_Jxy(Jxy)

Then, the structure matrices are accessed as follows:

In [13]:
print('M=')
core.M

M=


Matrix([
[0, -1, -1, -1],
[1,  0,  0,  0],
[1,  0,  0,  0],
[1,  0,  0,  0]])

In [14]:
print('J=')
core.J()

J=


Matrix([
[  0, -1.0, -1.0, -1.0],
[1.0,    0,    0,    0],
[1.0,    0,    0,    0],
[1.0,    0,    0,    0]])

In [15]:
print('Jxx=')
core.Jxx()

Jxx=


Matrix([
[  0, -1.0],
[1.0,    0]])

In [16]:
print('Jxw + Jwx.T =')
core.Jxw() + core.Jwx().T

Jxw + Jwx.T =


Matrix([
[0],
[0]])

In [17]:
print('R=')
core.R()

R=


Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### The `core.build_R` method
The resistive structure associated with linear dissipative components can be automatically derived as follows:

In [18]:
core.build_R()
core.R()

Matrix([[1]])


Matrix([
[1.0*R, 0, 0],
[    0, 0, 0],
[    0, 0, 0]])

## 5. Setting the physical parameters values
The correspondence between the parameters symbols defined above (here `L`, `C` and `R`) and their actual value is stored in the python dictionary `core.subs`, with parameters symbols as the dictionary's keys and numerical values as the dictionary's values. Here, the physical parameters are
* $L=50\times 10^{-3}$H, 
* $C=2\times 10^{-9}$F, 
* $R = 10^3\Omega$.

This is stored as follows:

In [19]:
L_value = 50e-3
C_value = 2e-9
R_value = 1e3

subs = {L: L_value,
        C: C_value,
        R: R_value}

core.subs.update(subs)

## Bonus: Generate the $\LaTeX$ description
A `.tex` file containing a description of the system can now be generated with the `core.texwrite` command as follows:

In [20]:
core.texwrite(filename=None, title=None)

A [core.tex](/pyphs_outputs/RLC/core.tex) has been generated, the compilation of which yields the following [core.pdf](/pyphs_outputs/RLC/core.pdf). A valid file name or title for the document can be specified.